In [1]:
import velodyne_decoder as vd
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def save_pcd(path, data, intensity):
    #convert = lambda e: [e.x, e.y, e.z]
    #data = np.array(list(map(convert, data)))
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(data)
    pcd.colors = o3d.utility.Vector3dVector(intensity)

    o3d.io.write_point_cloud(path, pcd)

def pcap2pcd_velodyne(pcap_file, model, out_dir, min_angle=0, max_angle=90):
    config = vd.Config(model=model, rpm=600, max_angle=max_angle, min_angle=min_angle)
    cloud_arrays = []
    for stamp, points in vd.read_pcap(pcap_file, config):
        cloud_arrays.append(points)

    cloud_arrays_np = np.array(cloud_arrays)

    cmap = plt.cm.winter

    for i in range(cloud_arrays_np.shape[0]):
        array = cloud_arrays_np[i]
        # array = array[array[:,3] >= 150,:]
        
        array_rgb = array[:,0:3]
        color = np.zeros(array_rgb.shape)
        color = cmap(array[:,3] / 255)[:,:3]
        print(array_rgb.shape, color.shape)
        save_pcd(out_dir + str(i) + ".pcd", array_rgb, color)

In [8]:
cloud_arrays_np = pcap2pcd_velodyne("0518kabe.pcap", "VLP-16", "out-0518/", min_angle=210, max_angle=270)

c:\Users\hykwy\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()


(4489, 3) (4489, 3)
(4496, 3) (4496, 3)
(4506, 3) (4506, 3)
(4517, 3) (4517, 3)
(4523, 3) (4523, 3)
(4505, 3) (4505, 3)
(4513, 3) (4513, 3)
(4486, 3) (4486, 3)
(4489, 3) (4489, 3)
(4492, 3) (4492, 3)
(4465, 3) (4465, 3)
(4435, 3) (4435, 3)
(4415, 3) (4415, 3)
(4390, 3) (4390, 3)
(4400, 3) (4400, 3)
(4386, 3) (4386, 3)
(4417, 3) (4417, 3)
(4445, 3) (4445, 3)
(4274, 3) (4274, 3)
(4254, 3) (4254, 3)
(4250, 3) (4250, 3)
(4241, 3) (4241, 3)
(4279, 3) (4279, 3)
(4241, 3) (4241, 3)
(4223, 3) (4223, 3)
(4245, 3) (4245, 3)
(4244, 3) (4244, 3)
(4253, 3) (4253, 3)
(4258, 3) (4258, 3)
(4257, 3) (4257, 3)
(4249, 3) (4249, 3)
(4246, 3) (4246, 3)
(4244, 3) (4244, 3)
(4240, 3) (4240, 3)
(4249, 3) (4249, 3)
(4258, 3) (4258, 3)
(4257, 3) (4257, 3)
(4244, 3) (4244, 3)
(4255, 3) (4255, 3)
(4247, 3) (4247, 3)
(4235, 3) (4235, 3)
(4237, 3) (4237, 3)
(4233, 3) (4233, 3)
(4251, 3) (4251, 3)
(4256, 3) (4256, 3)
(4249, 3) (4249, 3)
(4246, 3) (4246, 3)
(4263, 3) (4263, 3)
(4252, 3) (4252, 3)
(4253, 3) (4253, 3)


In [13]:
pcd = o3d.io.read_point_cloud("out-0518/150.pcd")

sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.01)

o3d.visualization.draw_geometries([pcd, sphere])

In [10]:
import glob
import os
import time

pcd_path = "./out-0518/"

frames = []
files = glob.glob(os.path.join(pcd_path, "*.pcd"))
for x in files:
    pcd = o3d.io.read_point_cloud(x)
    frames.append((pcd.points, pcd.colors))

vis = o3d.visualization.Visualizer()
vis.create_window()

frame_count = 0

pcd = o3d.geometry.PointCloud()
pcd.points = frames[frame_count][0]
pcd.colors = frames[frame_count][1]

vis.add_geometry(pcd)

sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.01)
sphere.paint_uniform_color(np.array([1.,0.,0.]))
vis.add_geometry(sphere)

frame_count += 1

slow_factor = 5

# time.sleep(5)

try:

    for index in range(len(frames)*slow_factor):
        # print(f"{frame_count}/{len(frames)}")
        pcd.points = frames[frame_count][0]
        pcd.colors = frames[frame_count][1]
        vis.update_geometry(pcd)
        vis.poll_events()
        vis.update_renderer()
        if index % slow_factor == 0:
            frame_count += 1
            if frame_count == len(frames):
                frame_count = 0

finally:
    vis.destroy_window()

# pcd.colorsに無理やり情報を詰め込む

In [ ]:
def save_pcd(path, data, intensity):
    #convert = lambda e: [e.x, e.y, e.z]
    #data = np.array(list(map(convert, data)))
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(data)
    pcd.colors = o3d.utility.Vector3dVector(intensity)

    o3d.io.write_point_cloud(path, pcd)

def pcap2pcd_velodyne(pcap_file, model, out_dir, min_angle=0, max_angle=90):
    config = vd.Config(model=model, rpm=600, max_angle=max_angle, min_angle=min_angle)
    cloud_arrays = []
    for stamp, points in vd.read_pcap(pcap_file, config):
        cloud_arrays.append(points)

    cloud_arrays_np = np.array(cloud_arrays)

    cmap = plt.cm.winter

    for i in range(cloud_arrays_np.shape[0]):
        array = cloud_arrays_np[i]
        
        array_xyz = array[:,0:3]
        color = np.zeros(array_xyz.shape)
        color[:,1] = array[:,3] / 255
        print(array_xyz.shape, color.shape)
        save_pcd(out_dir + str(i) + ".pcd", array_xyz, color)

In [ ]:
pcap2pcd_velodyne("vlp16-spoofing.pcap", "VLP-16", "out/muri/", min_angle=280, max_angle=310)